To save time you can run `set the environment` section and `read emeddings and index` session

# Set the Environment

In [1]:
!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install torch==2.1.0
!pip install openai==0.27.7

2023-12-07 20:59:21.284380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 20:59:21.284436: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 20:59:21.284474: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 20:59:21.292514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 20:59:23.091157: W tensorflow/c

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install gradio

In [4]:
!pip install -U sentence-transformers rank_bm25

In [5]:
# !pip install openai

In [6]:
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
import re
from tqdm import tqdm
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [7]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch



In [8]:
#show all columns
pd.set_option('display.max_columns', None)

In [9]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#TODO REMOVE CREDENTIALS
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="3dba0100c6bc49eab07f1622b0ca3413",
                                                           client_secret="ad818668808a42e69864bfc8a55cf6ae"))


In [10]:
!pip install faiss-gpu
import faiss

In [11]:
embedder = SentenceTransformer('all-mpnet-base-v2')
# embedder = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

# Read Data

In [12]:
df = pd.read_csv('/content/drive/MyDrive/spotify_millsongdata.csv')
df.shape

(57650, 4)

In [13]:
df.head()


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [14]:
df.describe()

,artist,song,link,text
count,57650,57650,57650,57650
unique,643,44824,57650,57494
top,Donna Summer,Have Yourself A Merry Little Christmas,/a/abba/ahes+my+kind+of+girl_20598417.html,I just came back from a lovely trip along the ...
freq,191,35,1,6


## data Preprocessing

In [15]:
# combine artist, song title, and lyrics
df["combined"] = (
    "Title: " + df.song.str.strip()+"; Artist: "+ df.artist.str.strip()
    + "; Lyrics: " + df.text.str.strip()

)

In [16]:
# remove special characters and transform into lowercase
df_combined = df.copy(deep=True)
df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))

In [17]:
df_combined.head()

,artist,song,link,text,combined
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",title ahes my kind of girl artist abba lyrics ...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",title andante andante artist abba lyrics take ...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,title as good as new artist abba lyrics ill ne...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,title bang artist abba lyrics making somebody ...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,title bangaboomerang artist abba lyrics making...


In [18]:
# remove carriage return and newline characters
df_combined.combined = df_combined.combined.str.replace('\r\n', '')

In [19]:
print('Longest str length is: ', df_combined.combined.str.len().max(), 'characters')

Longest str length is:  3879 characters


In [20]:
df_combined.iloc[df_combined.combined.str.len().idxmax()]['combined']

'title plex artist zro lyrics [zro]  they say he was flipping out till losing control of his mind  moving fast in a world that aint ready for the pace focussed on his grind  who am i to test zro i bet everybody in ridgemont 4 know me  feel what i say up under the evidence is still your homie  you must of forgot about the days until you be in 94  when it was 72 where partners sagging creases to find a hoe  now they cant look me in the eyes cause they might tell on theyself  about having something to do with trying to riddle my number one partner todeath  you want to get even with me boy clear you can see me with a rap  but im about gangsta shit so bow before you see me with a strap  cause ive been silent for so long its about time i broke the ice  nobody want to box me why everybody want to pull a trigger to take a life  im down with crews cause everybody else is fake to me  niggas with plex against me they all cakes to me  and everybody want to be talking about they real and they got l

As the lyrics length is relatively short, we don't need to apply chunk here.

In [21]:
# Use GPU
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

GPU Found!


# Embedding

In [ ]:
df_combined['combined'].shape

In [ ]:
# df_combined['embeddings'] = embedder.encode(df_combined['combined'])
sample = df_combined.iloc[:10,:]
sample = sample.copy()
sample.head()

In [ ]:
sample_embeddings = embedder.encode(sample['combined'])


In [ ]:
sample_embeddings.shape

In [ ]:
df_embeddings = embedder.encode(df_combined['combined'])

In [ ]:
df_embeddings.shape

## Store in FAISS

### Sample

In [ ]:
d = sample_embeddings.shape[1] # vector size
m = 32 # number of cells

index = faiss.IndexHNSWFlat(d, m)

index.train(sample_embeddings)
index.is_trained

In [ ]:
index.add(sample_embeddings)

In [ ]:
index.ntotal

In [ ]:
index.add(df_combined.embeddings)
index.ntotal

In [ ]:
%%time
k = 3 # number of returned songs
query = '''ABBA Ahe's My Kind Of Girl'''
q = embedder.encode([query])

distance, indices = index.search(q, k)

In [ ]:
sample.iloc[indices[0]]

### Whole Dataset

In [ ]:
d = df_embeddings.shape[1]
m = 32 # number of cells

index = faiss.IndexHNSWFlat(d, m)

index.train(df_embeddings)

index.add(df_embeddings)


In [ ]:
index.is_trained

# Read embeddings and index

In [ ]:
#np.savez('/content/drive/MyDrive/df_embeddings.npz', df_embeddings=df_embeddings) #save embeddings

In [22]:
df_embeddings = np.load('/content/drive/MyDrive/df_embeddings.npz')['df_embeddings']

In [23]:
# Save the index
#faiss.write_index(index, "/content/drive/MyDrive/hnsw_index.index")

# Load the index
index = faiss.read_index("/content/drive/MyDrive/hnsw_index.index")


In [24]:
index.ntotal

57650

## Use Llama2 for Keywords

In [25]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.


In [26]:
# !pip -qqq install bitsandbytes accelerate

In [27]:
# !pip install accelerate

In [28]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the bitsandbytes library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, you need an access token
hf_auth = '' # add your own access token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")


tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)


# stop_list = ['\nHuman:', '\n```\n']

# stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
# stop_token_ids


import torch
# stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
# stop_token_ids

from transformers import StoppingCriteria, StoppingCriteriaList

# # define custom stopping criteria object
# class StopOnTokens(StoppingCriteria):
#     def _call_(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         for stop_ids in stop_token_ids:
#             if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
#                 return True
#         return False

# stopping_criteria = StoppingCriteriaList([StopOnTokens()])


generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation', #text-generation
    # we pass model parameters here too
    # stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=4000,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)


from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1033: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [29]:
feeling = "I want to hear some sad music, I'm feeling bad"
print(llm(prompt=f"{feeling} from this sentence help me to think of 30 words that commonly used in songs to express the feeling of the person who said it"))

. everybody knows that music is a way to express our feelings and emotions, so if you are feeling sad or upset, listening to sad music can help you to feel better.

Here are 30 words that are commonly used in sad songs to express the feeling of sadness:

1. Tears
2. Pain
3. Sorrow
4. Heartache
5. Grief
6. Loss
7. Longing
8. Melancholy
9. Nostalgia
10. Regret
11. Bitterness
12. Disappointment
13. Betrayal
14. Loneliness
15. Isolation
16. Helplessness
17. Hopelessness
18. Desperation
19. Anguish
20. Torment
21. Suffering
22. Agony
23. Distress
24. Misery
25. Woe
26. Mourning
27. Gloom
28. Doom
29. Darkness
30. Despair

These words are often used in songs to convey the feeling of sadness and loss, and they can be found in many different genres of music, including pop, rock, country, and more.


## Final Search

### Without Llama2

In [30]:
def search_no_llama(query, k):
  """
  Return search results without Llama2
  query: user input
  k: number of returned songs
  """

  q = embedder.encode([query])
  distance, indices = index.search(q, k)

  return df_combined.iloc[indices[0]]

query = '''ABBA Ahe's My Kind Of Girl'''

In [31]:
query = "ABBA Ahe's My Kind Of Girl"
k = 3

In [32]:
search_no_llama(query, k)

,artist,song,link,text,combined
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",title ahes my kind of girl artist abba lyrics ...
34,ABBA,I Am Just A Girl,/a/abba/i+am+just+a+girl_20002700.html,"I am just a girl \r\nOne among the others, no...",title i am just a girl artist abba lyrics i am...
92,ABBA,That's Me,/a/abba/thats+me_20002861.html,Are you sure you want to hear more \r\nWhat i...,title thats me artist abba lyrics are you sure...


### With Llama2

In [113]:
def search_llama(query, k):
  """
  Return search results with Llama2 providing extra information
  query: user input
  k: number of returned songs
  """


  prompt = f"""{query} for this sentence, if it's lyrics from a song or it's a singer's name, give the artist name and the song's name. Otherwise,
   think of keywords that's commonly used in lyrics and are closely related to the sentence. Return only the list of songs OR keywords. Not both.
   Only give the answer without any chitchat."""

  keywords = llm(prompt=prompt)
  keywords = query + ' ' + keywords
  # print(keywords)

  q = embedder.encode([keywords]) # using the expanded keywords from chatgpt as actual query instead of user input
  distance, indices = index.search(q, k)

  return df_combined.iloc[indices[0]]

### discarded prompt

In [59]:
# def search_llama(query, k):
#   """
#   Return search results with Llama2 providing extra information
#   query: user input
#   k: number of returned songs
#   """


#   prompt = f"""{query} from this sentence help me to think of 30 words that commonly used in songs to that's closely related to the sentence. Return only the list of keywords without any further descriptions."""

#   keywords = llm(prompt=prompt)
#   keywords = query + ' ' + keywords
#   # print(keywords)

#   q = embedder.encode([keywords]) # using the expanded keywords from chatgpt as actual query instead of user input
#   distance, indices = index.search(q, k)

#   return df_combined.iloc[indices[0]]

### Test

In [34]:
print('---Results without Llama2---')
query = 'some music that you can listen to when you are sad'
search_no_llama(query, k)

---Results without Llama2---


,artist,song,link,text,combined
38020,John Denver,Like A Sad Song,/j/john+denver/like+a+sad+song_20073830.html,Usually in the morning I'm filled with sweet b...,title like a sad song artist john denver lyric...
47802,Pearl Jam,Happy When I'm Crying,/p/pearl+jam/happy+when+im+crying_20217935.html,E per noi un piacere augurarvi insieme \r\nAi...,title happy when im crying artist pearl jam ly...
39617,Keith Urban,Tonight I Wanna Cry,/k/keith+urban/tonight+i+wanna+cry_10156433.html,Alone in this house again tonight \r\nI got t...,title tonight i wanna cry artist keith urban l...


In [35]:
search_no_llama('songs by ABBA', k)

,artist,song,link,text,combined
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",title andante andante artist abba lyrics take ...
75,ABBA,Rikky Rock'n'Roller,/a/abba/rikky+rocknroller_20598408.html,Rikky Rock'n Roller now that you're the headli...,title rikky rocknroller artist abba lyrics rik...
78,ABBA,Rock 'n Roll Band,/a/abba/rock+n+roll+band_20002796.html,Sitting in the darkest corner \r\nIn the tend...,title rock n roll band artist abba lyrics sitt...


In [105]:
print('---Results with Llama2---')
query = 'some music that you can listen to when you are sad'
search_llama(query, k)

---Results with Llama2---


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,artist,song,link,text,combined
39617,Keith Urban,Tonight I Wanna Cry,/k/keith+urban/tonight+i+wanna+cry_10156433.html,Alone in this house again tonight \r\nI got t...,title tonight i wanna cry artist keith urban l...
47802,Pearl Jam,Happy When I'm Crying,/p/pearl+jam/happy+when+im+crying_20217935.html,E per noi un piacere augurarvi insieme \r\nAi...,title happy when im crying artist pearl jam ly...
38020,John Denver,Like A Sad Song,/j/john+denver/like+a+sad+song_20073830.html,Usually in the morning I'm filled with sweet b...,title like a sad song artist john denver lyric...
18341,Roy Orbison,Cry,/r/roy+orbison/cry_20119166.html,If your sweetheart sends a letter of goodbye ...,title cry artist roy orbison lyrics if your sw...
18959,Stevie Wonder,Smile Please,/s/stevie+wonder/smile+please_20131785.html,A smiling face is on earth like star \r\nA fr...,title smile please artist stevie wonder lyrics...
26239,Boney M.,Still I Am Sad,/b/boney+m/still+i+am+sad_20319125.html,See the stars come joining down from the sky ...,title still i am sad artist boney m lyrics see...
38769,Journey,I'm Cryin',/j/journey/im+cryin_20075652.html,"Oh, I'm cryin', tears are fallin' down. \r\nI...",title im cryin artist journey lyrics oh im cry...
5568,Evanescence,Only Happy When It Rains,/e/evanescence/only+happy+when+it+rains_205026...,Originally by Garbage \r\n \r\nI'm only happ...,title only happy when it rains artist evanesce...
30697,Dusty Springfield,No Sad Songs For Me,/d/dusty+springfield/no+sad+songs+for+me_20043...,(Tom Springfield) \r\n \r\nThe Springfields ...,title no sad songs for me artist dusty springf...
18342,Roy Orbison,Crying,/r/roy+orbison/crying_10244563.html,"I was all right for a while, I could smile for...",title crying artist roy orbison lyrics i was a...


In [120]:
search_llama('songs by ABBA', k)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,artist,song,link,text,combined
91,ABBA,Thank You For The Music,/a/abba/thank+you+for+the+music_20002662.html,"I'm nothing special, in fact I'm a bit of a bo...",title thank you for the music artist abba lyri...
75,ABBA,Rikky Rock'n'Roller,/a/abba/rikky+rocknroller_20598408.html,Rikky Rock'n Roller now that you're the headli...,title rikky rocknroller artist abba lyrics rik...
56,ABBA,Mamma Mia,/a/abba/mamma+mia_20002675.html,"[Chorus] \r\nMamma mia, here I go again \r\n...",title mamma mia artist abba lyrics [chorus] m...
78,ABBA,Rock 'n Roll Band,/a/abba/rock+n+roll+band_20002796.html,Sitting in the darkest corner \r\nIn the tend...,title rock n roll band artist abba lyrics sitt...
98,ABBA,The Way Old Friends Do,/a/abba/the+way+old+friends+do_20002573.html,You and I can share the silence \r\nFinding c...,title the way old friends do artist abba lyric...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,title bang artist abba lyrics making somebody ...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",title andante andante artist abba lyrics take ...
36,ABBA,"I Do, I Do, I Do, I Do, I Do",/a/abba/i+do+i+do+i+do+i+do+i+do_20002884.html,Love me or leave me \r\nMake your choice but ...,title i do i do i do i do i do artist abba lyr...
34,ABBA,I Am Just A Girl,/a/abba/i+am+just+a+girl_20002700.html,"I am just a girl \r\nOne among the others, no...",title i am just a girl artist abba lyrics i am...
96,ABBA,The Piper,/a/abba/the+piper_20002946.html,They came from the hills \r\nAnd they came fr...,title the piper artist abba lyrics they came f...


# Evaluation

## Queries for Offline Evaluation

In [91]:
query1 = 'songs by ABBA'
results1 = (df_combined[df_combined['artist']=='ABBA'][['artist', 'song']]).values.tolist()
results1

[['ABBA', "Ahe's My Kind Of Girl"],
 ['ABBA', 'Andante, Andante'],
 ['ABBA', 'As Good As New'],
 ['ABBA', 'Bang'],
 ['ABBA', 'Bang-A-Boomerang'],
 ['ABBA', 'Burning My Bridges'],
 ['ABBA', 'Cassandra'],
 ['ABBA', 'Chiquitita'],
 ['ABBA', 'Crazy World'],
 ['ABBA', 'Crying Over You'],
 ['ABBA', 'Dance'],
 ['ABBA', 'Dancing Queen'],
 ['ABBA', 'Disillusion'],
 ['ABBA', 'Does Your Mother Know'],
 ['ABBA', 'Dream World'],
 ['ABBA', 'Dum Dum Diddle'],
 ['ABBA', 'Eagle'],
 ['ABBA', 'Every Good Man'],
 ['ABBA', 'Fernando'],
 ['ABBA', 'Fernando (In Spanish)'],
 ['ABBA', 'Free As A Bumble Bee'],
 ['ABBA', 'From A Twinkling Star To A Passing Angel'],
 ['ABBA', 'Gimme Gimme Gimme'],
 ['ABBA', "Givin' A Little Bit More"],
 ['ABBA', 'Gonna Sing You My Lovesong'],
 ['ABBA', 'Hamlet III'],
 ['ABBA', 'Happy Hawaii'],
 ['ABBA', 'Happy New Year'],
 ['ABBA', 'He Is Your Brother'],
 ['ABBA', 'Head Over Heels'],
 ['ABBA', "Here We'll Stay"],
 ['ABBA', 'Hey Hey Helen'],
 ['ABBA', 'Hole In Your Soul'],
 ['ABBA

In [39]:
len(results1)

113

In [92]:
predicted1 = (search_llama(query1, 150)[['artist', 'song']]).values.tolist()
predicted1

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


songs by ABBA 

Example: "Love is in the air" - ABBA, "Dancing Queen"

Please provide the list of songs or keywords for each sentence.


[['ABBA', 'Reina Danzante'],
 ['ABBA', "Rikky Rock'n'Roller"],
 ['ABBA', 'Mamma Mia'],
 ['ABBA', 'Summer Night City'],
 ['ABBA', 'Nina, Pretty Ballerina'],
 ['ABBA', 'Dancing Queen'],
 ['ABBA', 'Thank You For The Music'],
 ['ABBA', 'Andante, Andante'],
 ['ABBA', 'Tropical Loveland'],
 ['ABBA', 'One Man, One Woman'],
 ['ABBA', 'So Long'],
 ['ABBA', "Rock 'n Roll Band"],
 ['ABBA', 'Kisses Of Fire'],
 ['ABBA', 'The Piper'],
 ['ABBA', 'Juper Jrouper'],
 ['ABBA', 'People Need Love'],
 ['ABBA', 'Honey, Honey'],
 ['ABBA', 'Fernando'],
 ['ABBA', 'Bang'],
 ['ABBA', 'We Wish You A Merry Christmas'],
 ['ABBA', 'SOS'],
 ['ABBA', 'Merry-Go-Round'],
 ['ABBA', "Love Has It's Ways"],
 ['ABBA', 'S. O. S.'],
 ['ABBA', 'Bang-A-Boomerang'],
 ['ABBA', 'Rock Me'],
 ['ABBA', 'The Way Old Friends Do'],
 ['ABBA', 'Rubber Ball Man'],
 ['ABBA', 'Lovelight'],
 ['ABBA', 'Gonna Sing You My Lovesong'],
 ['ABBA', 'Dance'],
 ['ABBA', 'I Let The Music Speak'],
 ['ABBA', 'On And On And On'],
 ['ABBA', 'Knowing Me Knowin

In [87]:
# query2 = """Is this the real life? Is this just fantasy? """
# results2 = (df_combined[df_combined['artist']=='Beach Boys'].iloc[3][['artist', 'song']]).values
# results2 = results2.reshape(1,-1)
# results2 = np.concatenate((results2, df_combined[df_combined['song']=='Bohemian Rhapsody'][['artist', 'song']].values)).tolist()
# results2

In [88]:
query2 = """Is this the real life? Is this just fantasy? Caught in a landslide No escape from reality
Open your eyes  Look up to the skies and see  I'm just a poor boy, I need no sympathy """

In [99]:
results2 = df_combined[df_combined['song']=='Bohemian Rhapsody'][['artist', 'song']].values
results2 = results2.reshape(1,-1).tolist()
results2

[['Queen', 'Bohemian Rhapsody']]

In [ ]:
# query2 = """Is this the real life? Is this just fantasy? """

In [95]:
predicted2 = (search_llama(query2, 20)[['artist', 'song']]).values.tolist()
predicted2

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[['Fleetwood Mac', 'Landslide'],
 ['Pearl Jam', 'Landslide'],
 ['Tori Amos', 'Landslide'],
 ['Olivia Newton-John', 'Landslide'],
 ['Glee', 'Landslide'],
 ['Queen', 'Bohemian Rhapsody'],
 ['Ace Of Base', 'Ravine'],
 ['Bon Jovi', 'Miracle'],
 ['Norah Jones', 'No Easy Way Down'],
 ['Orphaned Land', 'Mercy'],
 ['Weird Al Yankovic', 'Bohemian Polka'],
 ['Linkin Park', 'Morei Sky'],
 ['Verve', 'Country Song'],
 ['Dream Theater', 'About To Crash (Reprise)'],
 ['Nick Cave', 'Avalanche'],
 ['Pearl Jam', 'Parachutes'],
 ['Vertical Horizon', 'The Unchosen One'],
 ['America', "It's Life"],
 ['Pearl Jam', 'Down'],
 ['Uriah Heep', 'Sympathy']]

## Recall@K

In [96]:
def recall(actual, predicted, k):
    act_set = set(tuple(row) for row in actual)
    pred_set = set(tuple(row) for row in predicted[:k])
    result = len(act_set & pred_set) / float(len(act_set))
    return result

K = np.arange(1, 11)

for k in K:
  print(round(recall(results1, predicted1, k), 2))

0.01
0.02
0.03
0.04
0.04
0.05
0.06
0.07
0.08
0.09


In [100]:
for k in K:
  print(round(recall(results2, predicted2, k), 2))

0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0


## MRR

In [101]:
# To calculate MRR
def calculate_mrr(actual_list, predicted_list, k):
  """
  Return the list of rr for each query and the mrr for all queries
  actual_list: list of all actual results for different queries
  predicted_list: list of returned results for queries
  k: top k results
  """

  Q = len(actual_list)
  rr = []

  for actual_results, predicted_names in zip(actual_list, predicted_list):
    rank = 1000000
    # predicted_names = predicted_names.reset_index()
    for actual in actual_results:
        if actual in predicted_names:
          # print(predicted_names.index(actual))
          rank_temp = predicted_names.index(actual) + 1
            # print(rank_temp)
          rank = min(rank_temp, rank)
            # print(rank)

    rr.append(1 / rank)
    # print(rr)

  return rr, sum(rr) / Q

actual_list = [results1, results2]
predicted_list = [predicted1, predicted2]

rr, mrr = calculate_mrr(actual_list, predicted_list, 10)

# Print MRR for each query
print(f"RR for first query: {round(rr[0], 2)}")
print(f"RR for second query: {round(rr[1], 2)}")

print(f"MRR: {round(mrr, 2)}")

RR for first query: 1.0
RR for second query: 0.17
MRR: 0.58


## MAP

In [102]:
def calculate_map(actual_list, predicted_list, k):
  ap = []
  Q = len(actual_list)

  for q in range(Q):
    predicted_names = predicted_list[q]
    ap_num = 0
    k_temp = min(k, len(predicted_names))

    for x in range(k_temp):
      act_set = set(tuple(row) for row in actual_list[q])
      pred_set = set(tuple(row) for row in predicted_names[:x+1])
      precision_at_k = len(act_set & pred_set) / (x+1)
      if predicted_names[x] in actual_list[q]:
        rel_k = 1
      else:
        rel_k = 0
      # print(precision_at_k, rel_k)
      ap_num += precision_at_k * rel_k

    ap_q = ap_num / len(actual_list[q])
    print(f'AP@{k}_query{q+1} = {round(ap_q, 2)}')
    ap.append(ap_q)
  # print(ap, Q)
  map_at_k = sum(ap) / Q
  print(f"MAP@{k} = {round(map_at_k, 2)}")

  return map_at_k

calculate_map(actual_list, predicted_list, k)

AP@10_query1 = 0.09
AP@10_query2 = 0.17
MAP@10 = 0.13


0.1275811209439528

# Final UI

## Spotify Helper

In [103]:
def get_track_url(q, limit):
    """
    Hook the search engine with Spotify API
    q: user input
    limit: number of returned results
    """

    # df1 = search_no_llama(q, limit*3)
    # df2 = search_llama(q, limit*3)
    # df = pd.concat([df1, df2], ignore_index=True)
    # df = df.drop_duplicates()
    # # df = df.sample(frac=1, random_state=42)
    # df = df.iloc[:limit]
    df = search_llama(q, limit)

    results = (df['artist'] +' ' + df['song']).to_list() # get the list of songs returned from the search engine

    tracks = []
    for song in results: # retrieve the songs from Spotify API
      sp_song = sp.search(song, 1, offset=0, type='track', market=None)
      items = sp_song['tracks']['items']
      for item in items:
          tracks.append({
              'name': item['name'],
              'artist': item['artists'][0]['name'],
              'url': item['external_urls']['spotify'],
              'image_url': item['album']['images'][0]
          })

    return tracks



In [49]:
get_track_url('dancing songs', 5)

[{'name': "Look Who's Dancing",
  'artist': 'Ziggy Marley & The Melody Makers',
  'url': 'https://open.spotify.com/track/2clwOnKvcZ1mG5IOqXVTuz',
  'image_url': {'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b2730be84b9d3f8aaa548d464829',
   'width': 640}},
 {'name': 'On the Dancefloor (feat. will.i.am & apl.de.ap)',
  'artist': 'David Guetta',
  'url': 'https://open.spotify.com/track/0cBom2A1RpP15quVF5UVDD',
  'image_url': {'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273b2e43ed3850e072831f7b2c6',
   'width': 640}},
 {'name': 'Keep on Dancing',
  'artist': 'Chris Rea',
  'url': 'https://open.spotify.com/track/555adBEPF5sKqBRoeP2218',
  'image_url': {'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b2739c8d257b1af6187595e7c087',
   'width': 640}},
 {'name': 'The Loco-Motion - Remastered 2002',
  'artist': 'Grand Funk Railroad',
  'url': 'https://open.spotify.com/track/0PxYWaGiwWtcuL1VIMiEW4',
  'image_url': {'height': 640,
   'url': 'https

## Gradio Helper

In [121]:
import gradio as gr
k = 10 # number of songs returned


def search(Query):

  result = get_track_url(Query, k)

  print(result)

  header_html = '<h2>Play List</h2>'

  tbl_header_html = """
        <table border="1" style="width:100%">
        <tr>
            <th style="width:50%">Artist</th>
            <th style="width:50%">Song</th>
        </tr>
                    """
  tbl_rows_html = """"""

  for r in result:
    tbl_row_html = """
        <tr>
            <td>{0}</td>
            <td><a href="{1}" target="_blank">{2}</a></td>
        </tr>
        """.format(r['artist'], r['url'], r['name'])

    tbl_rows_html += tbl_row_html

  tbl_end_html = """</table>"""

  html = header_html + tbl_header_html + tbl_rows_html + tbl_end_html


  return [html]


iface = gr.Interface(fn=search, inputs="text", outputs="html", theme='JohnSmith9982/small_and_pretty', css="footer{display:none !important}", allow_flagging='never')
iface.feedback_component = None

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0303f34d6e170698e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
